In [7]:
from lap_stroke_pipeline import (
    run_pipeline_from_db,
    load_from_db,
    add_accel_combined,
    add_gyro_combined,
    add_is_swimming,
    clean_is_swimming,
    LapConfig,
    detect_laps_from_is_swimming,
    identify_stroke_cycles,
)
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.rcParams["figure.figsize"] = (12, 4)

# ---- CONFIG ----
DB_PATH = r"test-may20-w1-breaststroke.db"  # change if needed
TABLE_NAME = "sensor_data"

# Lap detection parameters (same as pipeline defaults)
lap_cfg = LapConfig(
    accel_threshold=12.0,
    gap_fill_seconds=7.0,
    bout_filter_seconds=30.0,
)

In [8]:
per_lap, session_avgs = run_pipeline_from_db(
    db_path=DB_PATH,
    table_name=TABLE_NAME,
    stroke_type_col="stroke_type",  # ignored if column missing
    lap_config=lap_cfg,
)

per_lap_df = pd.DataFrame(per_lap)
per_lap_df

,lap_number,start_time,end_time,lap_time,stroke_count,stroke_type,velocity,stroke_rate_s,stroke_rate_min,stroke_length,stroke_index
0,1,2025-05-20 07:13:07.495000+08:00,2025-05-20 07:13:54.907000+08:00,47.412,18,None,1.054585,0.379651,22.779043,2.777778,2.929404
1,2,2025-05-20 07:14:09.628000+08:00,2025-05-20 07:14:54.467000+08:00,44.839,18,None,1.115101,0.401436,24.086175,2.777778,3.097502
2,3,2025-05-20 07:15:15.513000+08:00,2025-05-20 07:15:55.598000+08:00,40.085,16,None,1.247349,0.399152,23.949108,3.125000,3.897967


In [9]:
stroke_type_options = [
    "Freestyle",
    "Backstroke",
    "Breaststroke",
    "Butterfly",
    "IM",
    "Other",
]

lap_df = per_lap_df.copy()
lap_df["stroke_type"] = None

lap_select = widgets.Dropdown(
    options=sorted(lap_df["lap_number"].tolist()),
    description="Lap #",
    style={"description_width": "initial"},
)

stroke_select = widgets.Dropdown(
    options=stroke_type_options,
    description="Stroke type",
    style={"description_width": "initial"},
)

apply_button = widgets.Button(
    description="Set stroke type for lap",
    button_style="success",
)

table_output = widgets.Output()


def refresh_table():
    with table_output:
        clear_output()
        df_disp = lap_df.copy()
        for col in ["lap_time", "velocity", "stroke_rate_s", "stroke_rate_min", "stroke_length", "stroke_index"]:
            df_disp[col] = df_disp[col].round(3)
        display(df_disp)


def on_apply_clicked(_):
    lap_num = lap_select.value
    stype = stroke_select.value
    lap_df.loc[lap_df["lap_number"] == lap_num, "stroke_type"] = stype
    refresh_table()


apply_button.on_click(on_apply_clicked)

display(widgets.HBox([lap_select, stroke_select, apply_button]))
refresh_table()

In [10]:
print("Session averages:")
print(f"  avg_lap_time:     {session_avgs['avg_lap_time']:.2f} s")
print(f"  avg_stroke_count: {session_avgs['avg_stroke_count']:.1f} strokes")
print(f"  avg_velocity:     {session_avgs['avg_velocity']:.3f} m/s")
print(f"  avg_stroke_rate:  {session_avgs['avg_stroke_rate']:.3f} Hz ({session_avgs['avg_stroke_rate']*60:.1f} spm)")
print(f"  avg_stroke_length:{session_avgs['avg_stroke_length']:.3f} m/stroke")
print(f"  avg_stroke_index: {session_avgs['avg_stroke_index']:.3f} m^2/s")

Session averages:
  avg_lap_time:     44.11 s
  avg_stroke_count: 17.3 strokes
  avg_velocity:     1.139 m/s
  avg_stroke_rate:  0.393 Hz (23.6 spm)
  avg_stroke_length:2.894 m/stroke
  avg_stroke_index: 3.308 m^2/s


In [11]:
df_full = load_from_db(DB_PATH, table_name=TABLE_NAME)
add_accel_combined(df_full)
add_gyro_combined(df_full)
add_is_swimming(df_full, lap_cfg)
clean_is_swimming(df_full, lap_cfg)

In [12]:
def show_lap(lap_number: int):
    row = lap_df.loc[lap_df["lap_number"] == lap_number].iloc[0]
    start_time = row["start_time"]
    end_time = row["end_time"]

    mask = (df_full["datetime"] >= start_time) & (df_full["datetime"] <= end_time)
    segment = df_full.loc[mask].copy()

    print(f"Lap {lap_number} ({row['stroke_type']}): {start_time} → {end_time}, rows={len(segment)}")

    stroke_count, peaks, signal = identify_stroke_cycles(segment)
    print(f"Detected strokes in this lap: {stroke_count}")

    plt.figure(figsize=(12, 4))
    plt.plot(segment["datetime"], signal, label="Filtered ay+az (stroke signal)")
    if peaks:
        plt.plot(segment["datetime"].iloc[peaks], signal[peaks], "rx", label="Detected strokes")
    plt.xlabel("Time")
    plt.ylabel("Filtered Accel (m/s^2)")
    plt.title(f"Lap {lap_number} – stroke detection")
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)
    plt.plot(segment["datetime"], segment["accel_combined"], label="Accel Combined", color="blue")
    plt.ylabel("Combined Accel (g)")
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.plot(
        segment["datetime"],
        segment["is_swimming_cleaned"],
        label="is_swimming_cleaned",
        color="green",
        drawstyle="steps-post",
    )
    plt.xlabel("Time")
    plt.ylabel("Is Swimming (cleaned)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


lap_plot_dropdown = widgets.Dropdown(
    options=sorted(lap_df["lap_number"].tolist()),
    description="Plot lap #",
    style={"description_width": "initial"},
)
plot_button = widgets.Button(
    description="Show lap plots",
    button_style="info",
)
plot_output = widgets.Output()


def on_plot_clicked(_):
    with plot_output:
        clear_output()
        show_lap(lap_plot_dropdown.value)


plot_button.on_click(on_plot_clicked)

display(widgets.HBox([lap_plot_dropdown, plot_button]), plot_output)

Output()